# Guided Project: Building a Database for Crime Reports

In this guided project, we will put everything together to build a database for storing data related with crimes that occurred in Boston. This dataset is available in the file <code>boston.csv</code>.

The goal of this guided project is for you to create a database named <code>crimes_db</code> with a table – <code>boston_crimes</code> – with appropriate datatypes for storing the data from the <code>boston.csv</code> file. You will be creating the table inside a schema named <code>crimes</code>. You will also create the <code>readonly</code> and <code>readwrite</code> groups with the appropriate privileges as we have discussed in this course. Finally, you will also need to create one user for each of these groups.

The following diagram illustrates a high level overview of what we want to achieve:

<img src="Guided Project - Building a database for crime reports img 1.png" width=600px>

In [1]:
import csv
import pandas as pd
import psycopg2

In [2]:
# first few lines in the boston.csv file
with open("boston.csv", "r") as f:
    reader = csv.reader(f)
    data = list(reader)
    for row in data[:5]:
        print(row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']
['2', '1402', 'VANDALISM', '2018-08-21', 'Tuesday', '42.30682138', '-71.06030035']
['3', '3410', 'TOWED MOTOR VEHICLE', '2018-09-03', 'Monday', '42.34658879', '-71.07242943']
['4', '3114', 'INVESTIGATE PROPERTY', '2018-09-03', 'Monday', '42.33418175', '-71.07866441']


Create the database that will house the information in Postgres database. Need to set the <code>connection.autocommit</code> to <code>True</code> to create the intial database.

In [3]:
sql_text = """
           CREATE DATABASE crime_db; 
           """
conn = psycopg2.connect(dbname="postgres", user="postgres", password="abc123")
conn.autocommit = True
cur = conn.cursor()
cur.execute(sql_text)
conn.close()

Create <code>sql_execute</code> to connect to the new <code>crime_db</code> database on my computer and run the SQL commands.

In [4]:
def sql_execute(sql_text, autocommit=False, return_results=False):
    conn = psycopg2.connect(dbname="crime_db", user="postgres", password="abc123")
    conn.autocommit = autocommit
    cur = conn.cursor()

    cur.execute(sql_text)
    
    if return_results == True:
        data = cur.fetchall()

    conn.commit()
    conn.close()
    
    if return_results == True:
        return data

Create the schema holder.

In [5]:
sql_text = """
           CREATE SCHEMA crimes; 
           """
sql_execute(sql_text)

Now we have a database and a schema, lets look at data we import from the <code>boston.csv</code> above. We will read the data into a pandas dataframe and look at the unique values in each column. This was we can decide was the appropriate data types each column in our database needs to be.

In [6]:
df = pd.DataFrame(data[1:], columns=data[0])
df.set_index("incident_number", inplace=True)
df

,offense_code,description,date,day_of_the_week,lat,long
incident_number,,,,,,
1,619,LARCENY ALL OTHERS,2018-09-02,Sunday,42.35779134,-71.13937053
2,1402,VANDALISM,2018-08-21,Tuesday,42.30682138,-71.06030035
3,3410,TOWED MOTOR VEHICLE,2018-09-03,Monday,42.34658879,-71.07242943
4,3114,INVESTIGATE PROPERTY,2018-09-03,Monday,42.33418175,-71.07866441
5,3114,INVESTIGATE PROPERTY,2018-09-03,Monday,42.27536542,-71.09036101
...,...,...,...,...,...,...
298325,3125,WARRANT ARREST,2016-06-05,Sunday,42.33695098,-71.08574813
298326,111,"MURDER, NON-NEGLIGIENT MANSLAUGHTER",2015-07-09,Thursday,42.25592648,-71.12317207
298327,3125,WARRANT ARREST,2015-07-09,Thursday,42.25592648,-71.12317207


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 298329 entries, 1 to 298329
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   offense_code     298329 non-null  object
 1   description      298329 non-null  object
 2   date             298329 non-null  object
 3   day_of_the_week  298329 non-null  object
 4   lat              298329 non-null  object
 5   long             298329 non-null  object
dtypes: object(6)
memory usage: 15.9+ MB


Now we will look at the number of unique values in each column and decide what the proper datatypes should be for each column.

In [8]:
unique = {}

for col in df.columns:
    unique[col] = df[col].unique()

print("Unique Values For Each Column\n")
print("{:.<16}{:.>15}".format("column","unique values"))
for val in unique:
    print("{:.<16}{:.>15}".format(val,len(unique[val])))

Unique Values For Each Column

column............unique values
offense_code................219
description.................239
date.......................1177
day_of_the_week...............7
lat.......................18177
long......................18177


In [9]:
print("Longest Number of Characters in Each Column")
print("{:.<16}{:.>15}".format("column","max_len"))
for val in unique:
    max_len = max([len(x) for x in unique[val]])
    print("{:.<16}{:.>15}".format(val,max_len))
    for row in unique[val]:
        if len(row) == max_len:
            print("{}\n".format(row))
            break

Longest Number of Characters in Each Column
column..................max_len
offense_code..................4
1402

description..................58
RECOVERED - MV RECOVERED IN BOSTON (STOLEN OUTSIDE BOSTON)

date.........................10
2018-09-02

day_of_the_week...............9
Wednesday

lat..........................11
42.35779134

long.........................12
-71.13937053



Now that we have some information about the data in each column we can select the specific data types that will be used for each column. So the offense_code is a 4 character number. description is a text block with up to 58 characters. The data column has date values in the year-month-day formate. day_of_the_week is the full word of the day. Both lat and long or 8 digit float values with long being negative.

|column|datatype|
|:---|:---:|
|incident_number|integer PRIMARY KEY|
|offense_code|smallint|
|description|varchar(100)|
|date|date|
|day_of_the_week|enumerated|
|lat|numeric(11,8)|
|long|numeric(11,8)|

In [10]:
# Create enumerated datatype
sql_text = """
           CREATE TYPE day_enum AS ENUM ('Sunday', 'Monday', 'Tuesday', 'Wednesday',
           'Thursday', 'Friday', 'Saturday'); 
           """
sql_execute(sql_text)

In [11]:
# Create table
sql_text = """
           CREATE TABLE crimes.crime_reports (
           incident_number integer PRIMARY KEY,
           offense_code SMALLINT NOT NULL,
           description VARCHAR(100) NOT NULL,
           date DATE NOT NULL,
           day_of_the_week day_enum NOT NULL,
           lat NUMERIC(11, 8),
           long numeric(11, 8)
           ); 
           """
sql_execute(sql_text)

Now that we have create the table will will insert into the table values from <code>boston.csv</code>. Will be using the <code>cursor.copy_expert</code> within the <code>psycopg2</code> pacakge. 

In [12]:
# Inserting in data to crime_reports
conn = psycopg2.connect(dbname="crime_db", user="postgres", password="abc123")
cur = conn.cursor()

with open("boston.csv", "r") as f:
    cur.copy_expert("COPY crimes.crime_reports FROM STDIN WITH CSV HEADER", f)

conn.commit()
conn.close()

Now we have the data inserted we can start handing users. Our goal is to create the two user groups that we have learned about: <code>readonly</code> and <code>readwrite</code>. By following the least privilege principle, the first step in doing so is to make sure that there are no privileges inherited from the <code>public</code> group and on the <code>public</code> schema.

In [13]:
sql_text = """
           REVOKE ALL ON SCHEMA public FROM public;
           REVOKE ALL ON DATABASE crime_db FROM public;
           """
sql_execute(sql_text)

We have made sure that we are not going to inadvertently inherit privileges from the <code>public</code> group. The next step is to create our two users groups. That the <code>readonly</code> group is supposed to only privileges to perform <code>SELECT</code> queries. In contrast, we want the <code>readwrite</code> group to be able to perform <code>SELECT, INSERT, DELETE</code> and <code>UPDATE</code> queries. To grant <code>INSERT</code> and <code>SELECT</code> privileges in all tables inside a schema we can use the command

In [14]:
# Create and set access for readonly group
# Set and set access for readwrite
sql_text = """
           CREATE GROUP readonly NOLOGIN;
           GRANT SELECT ON ALL TABLES IN SCHEMA crimes to readonly;
           CREATE GROUP readwrite NOLOGIN;
           GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes to readwrite;
           """
sql_execute(sql_text)

Now that we have the groups setup for the database we can create a <code>data_analyist</code> user and set their groups as <code>readonly</code> and a <code>data_scientist</code> user and set their groups as <code>readwrite</code>.

In [15]:
sql_text = """
           CREATE USER data_analyist WITH PASSWORD 'secret1';
           GRANT readonly TO data_analyist;
           CREATE USER data_scientist WITH PASSWORD 'secret2';
           GRANT readwrite TO data_scientist;
           """
sql_execute(sql_text)

Now we can check privileges for users.

In [16]:
sql_text = """
           SELECT grantee, privilege_type
           FROM information_schema.table_privileges
           WHERE grantee = 'readwrite';
           """
sql_execute(sql_text, return_results=True)

[('readwrite', 'INSERT'),
 ('readwrite', 'SELECT'),
 ('readwrite', 'UPDATE'),
 ('readwrite', 'DELETE')]

In [17]:
sql_text = """
           SELECT grantee, privilege_type
           FROM information_schema.table_privileges
           WHERE grantee = 'readonly';
           """
sql_execute(sql_text, return_results=True)

[('readonly', 'SELECT')]